In [1]:
import numpy as np
from scipy.stats import norm
import numba as nb
from numba import njit, prange


@njit
# Define the payoff function for a call option
def payoff(S, K):
    return np.maximum(S - K, 0)

@njit(fastmath=True,parallel=True)
def PDE(S, K, r, sigma,T, M, N):
    # Define the space and time grids
    S_max = 10.0 * K # upper limit for the asset value
    ds = S_max / M # space step
    dt = T / N # time step
    S_grid = np.linspace(0, S_max, M+1) # space grid
    t_grid = np.linspace(0, T, N+1) # time grid
    

    # Initialize the option value matrix
    V = np.zeros((M+1, N+1))

    # Set the terminal condition (payoff at maturity)
    V[:, -1] = payoff(S_grid, K)

    
    # Set the boundary conditions (option value at S=0 and S=S_max)
    V[0, :] = 0.0                              # option value is zero when asset value is zero
    V[-1, :] = S_max - K * np.exp(-r * t_grid) # option value when asset value is very large

    # Loop backwards in time to calculate the option value at each node
    for n in range(N-1, -1, -1): # from N-1 to 0
        for m in range(1, M): # from 1 to M-1
            # Calculate the coefficients a, b and c for the finite difference scheme
            sig = sigma**2 * m**2
            a = 0.5 * dt * (sig - r * m)
            b = 1 - dt   * (sig + r)
            c = 0.5 * dt * (sig + r * m)
            # Update the option value at node (m, n) using the explicit scheme
            V[m, n] = a * V[m-1, n+1] + b * V[m, n+1] + c * V[m+1, n+1]

    ## Interpolate the option value at S using linear interpolation
    #i = int(np.floor(S / ds)) # find the index of the closest node below S
    #alpha = (S - S_grid[i]) / ds # find the interpolation weight
    #V_S = alpha * V[i+1, 0] + (1 - alpha) * V[i, 0] # interpolate the option value
    
    
    return np.interp(S, S_grid, V[:,0])
    #return V_S

In [2]:
# Define the parameters
S = 20.0 # initial asset value
K = 10.0 # strike price
r = 0.1 # risk-free interest rate
sigma = 0.4 # volatility
T = 0.25 # time to maturity
M = 350 # number of space intervals
N = 5000 # number of time intervals

%timeit PDE(S, K, r, sigma,T, M, N)
V_S =  PDE(S, K, r, sigma,T, M, N)
# Print the result
print("The European call option value is {:.4f}".format(V_S))

8.02 ms ± 322 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The European call option value is 10.2470


In [3]:
import math
# Define the Black-Scholes formula for a call option
def black_scholes(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

# Calculate the option value
V = black_scholes(S, K, T, r, sigma)

# Print the result
print("The European call option value is {:.4f}".format(V))


The European call option value is 10.2470


In [4]:
V_S-V

6.34006878463822e-06

In [5]:
import numpy as np
from scipy.stats import norm
import numba as nb
from numba import njit, prange

N = 2 # number of time intervals
M = int(4e7) # number of simulations

@njit(fastmath=True,parallel=True)
def MC_numba(S, K, r, sigma,T, M, N):

    # Define the time and space grids
    dt = T / N # time step
    S_grid = np.zeros((M, N+1)) # space grid
    S_grid[:, 0] = S # initial condition
    S_grid2 = np.zeros((M, N+1)) # space grid
    S_grid2[:, 0] = S # initial condition

    # Loop over the simulations and time steps to update the asset price
    r_sig_dt = (r - 0.5 * sigma**2) * dt
    sigma_sqrt_dt = sigma * np.sqrt(dt)
    for i in prange(M):
        for j in range(N):
            Z = np.random.normal()
            S_grid[i, j+1] = S_grid[i, j] * np.exp(r_sig_dt + sigma_sqrt_dt * Z)
            S_grid2[i, j+1] = S_grid2[i, j] * np.exp(r_sig_dt - sigma_sqrt_dt * Z)
    # Calculate the payoff of the option at maturity for each simulation
    payoff = 0.5*( np.maximum(S_grid[:, -1] - K, 0) + np.maximum(S_grid2[:, -1] - K, 0) )

    # Discount the payoff to get the present value and take the average
    V = np.exp(-r * T) * np.mean(payoff)
    return V

In [6]:
%timeit -r 1 MC_numba(S, K, r, sigma,T, M, N)
V_MC = MC_numba(S, K, r, sigma,T, M, N)
# Print the result
print("The European call option value is {:.4f}".format(V_MC))

4.47 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
The European call option value is 10.2471


In [7]:
# Implements for a european call option based on 
# https://hamedhelali.github.io/blog-post/Binomial-lattice/
import numpy as np

@njit(fastmath=True,parallel=True)
def binomial_lattice(S_0, K, sigma, r, T, M):

    dt = T / M
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    discount = np.exp(-r * dt)
    p_u = discount * p
    p_d = discount * (1-p)

    SVals = np.zeros(2*M+1)
    PVals = np.zeros(2*M+1)

    SVals[0] = S_0 * d**M

    for i in range(1,2*M+1):
        SVals[i] = SVals[i-1] * u

    for j in range(M+1):
        i = 2*j 
        PVals[i] = np.maximum(SVals[i]-K, 0)
        
    for tau in range(0, M):
        for j in range(0, M-tau):
            i = 2*j + tau + 1 
            PVals[i] = p_u * PVals[i+1] + p_d * PVals[i-1]

    return PVals[M]

S_0 = 20.0
K = 10.0
sigma = 0.4
r = 0.10
T = 0.25
M = 100


In [8]:
%timeit binomial_lattice(S_0, K, sigma, r, T, M)
print(binomial_lattice(S_0, K, sigma, r, T, M))

19.1 µs ± 6.93 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.247000290904706
